In [32]:
# Downloading necessary dependencies
! pip install spacy pandas openpyxl
! python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 11.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [33]:
import spacy
import pandas as pd

# Excel-Datei lesen
df = pd.read_excel(r'Stellenausschreibungen_2022_HES.xlsx')


In [34]:
df.describe().T

,count,unique,top,freq
Titel,626,563,Data Analyst (m/w/d),18
Unternehmen,626,503,Volkswagen AG,5
Text,626,623,Stellenbeschreibung\nAnstellungsart\nVollzeit\...,2
Unternehmen - 1. Buchstabe,626,30,A,73
6 Gruppen,6,6,Ende Teil 1,1
5 Gruppen,4,4,Ende Teil 1,1


In [35]:
df.head

<bound method NDFrame.head of                                                  Titel  \
0    (Junior) Controller - Financial Planning & Ana...   
1                           (Junior) Sales Ops Analyst   
2                         Web Analytics Expert (m/w/d)   
3              Data Analysis / Datenauswertung (m/w/d)   
4         Lead Data Analyst/Consultant (m/w/d), Berlin   
..                                                 ...   
621  (Senior) Data Analyst / Data Manager (w/m/d) C...   
622           Senior Consultant Data Analytics (m/w/d)   
623     (Senior) Business Analyst (m/w/d) Credit Lines   
624              Business Analyst (m/w/d) Credit Lines   
625                   Service Business Analyst (m/w/d)   

                   Unternehmen  \
0                         1NCE   
1                         1NCE   
2                  21TORR GmbH   
3    3P Services GmbH & Co. KG   
4     9 friendly white rabbits   
..                         ...   
621         ZEIT Verlagsgruppe   
6

In [36]:
# Lade das deutsche Spacy-Modell und die Stopwörter-Liste
nlp = spacy.load("de_core_news_sm", disable=["tagger", "parser", "ner"])
stopwords = nlp.Defaults.stop_words

# Methoden zur Tokenisierung mit und ohne stopwords
def tokenize_text(text):
    doc = nlp(text)
    return [token.text for token in doc]

def tokenize_without_stopwords(text):
    doc = nlp(text)
    return [token.text for token in doc if not (token.is_stop or token.is_punct)]

def tokenize_only_stopwords(text):
    doc = nlp(text)
    return [token.text for token in doc if token.is_stop]

In [39]:
# Analyse der Spalte "Text"
df["tokens"] = df["Text"].apply(tokenize_text)
df["tokens_without_stopwords"] = df["Text"].apply(tokenize_without_stopwords)
df["tokens_stopwords_only"] = df["Text"].apply(tokenize_only_stopwords)

df["without_stopwords"] = df["tokens_without_stopwords"].apply(lambda x: " ".join(x))
df["stopwords_only"] = df["tokens_stopwords_only"].apply(lambda x: " ".join(x))

# Analyse: Hier die Anzahl der Wörter ohne Stoppwörter und nur mit Stoppwörtern
df["count_without_stopwords"] = df["tokens_without_stopwords"].apply(len)
df["count_stopwords"] = df["tokens_stopwords_only"].apply(len)

# Ermittlung der fünf am häufigsten verwendeten Wörter ohne Stopwörter
all_words_without_stopwords = [word for sublist in df["tokens_without_stopwords"].tolist() for word in sublist if word.strip()]
word_freq = Counter(all_words_without_stopwords)
most_common_words = word_freq.most_common(7)  # Hole die 7 häufigsten, da wir möglicherweise 2 ignorieren müssen

# Filtere die leeren Zeichenketten und Satzzeichen aus der meistverwendeten Liste heraus
filtered_most_common_words = [item for item in most_common_words if item[0].strip() and not all(char in ':;,.-?!' for char in item[0])][:5]

print("Die fünf am häufigsten verwendeten Wörter (ohne Stopwörter) sind:")
for word, freq in filtered_most_common_words:
    print(f"{word}: {freq} mal")
    
# Speichern derErgebnisse in einer neuen Excel-Datei
df.to_excel("Stellenausschreibungen_2022_HES-solutions.xlsx", index=False)

print("Analyse und Tokenisierung abgeschlossen und Ergebnisse gespeichert.")

Die fünf am häufigsten verwendeten Wörter (ohne Stopwörter) sind:
Data: 1084 mal
Stellenbeschreibung: 925 mal
Kunden: 800 mal
Analytics: 785 mal
Team: 709 mal
Analyse und Tokenisierung abgeschlossen und Ergebnisse gespeichert.
